In [1]:
# Fix
from run import main
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [ ]:
best_models = main(log=True)

# Deleted the existing log file.

# LOADING the data...
    >>> SUCCESS!

# Converting the data into preconfigured dtypes...
    >>> SUCCESS!

#########################################################

SUMMARY for the loaded dataset

The total number of rows is 124976

The total number of rows is 25

Descriptive Statistics:

       school_ncesid  school_latitude  school_longitude  \
count  1.157430e+05   124976.000000    124976.000000      
mean   2.448448e+11   36.827284       -95.859299          
std    1.644728e+11   4.963669         18.392876          
min    1.000050e+10   18.249140       -171.690554         
25%    6.344101e+10   33.872504       -117.806418         
50%    2.200870e+11   36.617410       -90.101563          
75%    3.704880e+11   40.676156       -80.713740          
max    6.100010e+11   65.672562       -66.628036          

       total_price_including_optional_support  students_reached  
count  124976.000000                           124917.000000     
mean   65

In [ ]:
train_df = temporal_sets[2][1][0]
test_df = temporal_sets[2][1][1]

In [ ]:
len(temporal_sets[2][1][2])

In [ ]:
#print(test_df['date_posted'].min())
#print(test_df['date_posted'].max())
#print(train_df['date_posted'].min())
#print(train_df['date_posted'].max())

In [ ]:
test_df

In [ ]:
#test_df = test_df.reset_index()
#test_df = test_df.drop(['projectid', 'date_posted', 'datefullyfunded'], axis=1)

In [ ]:
#test_df.shape

In [ ]:
#ValueError: operands could not be broadcast together with shapes (44167,68) (67,) (44167,68)